# cstore_suite Colab 런처

이 노트북은 `cstore_suite` 프로젝트를 Colab에서 한 번에 실행하기 위한 런처입니다.

순서대로 위에서 아래로 셀만 실행하면 됩니다.


In [ ]:
%cd /content/cstore_suite

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-693870770.py", line 1, in <cell line: 0>
    get_ipython().run_line_magic('cd', '/content/cstore_suite')
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py", line 2418, in run_line_magic
    result = fn(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^
  File "<decorator-gen-85>", line 2, in cd
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/magic.py", line 187, in <lambda>
    call = lambda f, *a, **k: f(*a, **k)
                              ^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/magics/osm.py", line 342, in cd
    oldcwd = os.getcwd()
             ^^^^^^^^^^^
OSError: [Errno 107] Transport endpoint is not connected

During handling of the above exception, another exception occurred:

Traceb

In [7]:
# 1️⃣ Colab / Jupyter 환경에서 cstore_suite 프로젝트 폴더 자동 설정
import os, sys

print("✅ Python version:", sys.version)

# (선택) Google Colab 에서 실행 중이면 Drive 마운트 시도
IN_COLAB = False
try:
    from google.colab import drive  # type: ignore
    IN_COLAB = True
except Exception:
    IN_COLAB = False

if IN_COLAB:
    try:
        drive.mount("/content/drive")
    except Exception as e:
        print("⚠ Drive 마운트 중 경고:", e)

# 프로젝트 위치 후보들
candidates = []
candidates.append("/content/cstore_suite")  # Colab에서 ZIP 풀어서 쓴 경우
candidates.append("/content/drive/MyDrive/cstore_suite")  # Drive에 폴더가 있는 경우
candidates.append(os.getcwd())  # 현재 폴더

proj = None
for p in candidates:
    if os.path.isdir(p):
        proj = p
        break

if proj is None:
    raise FileNotFoundError(
        "cstore_suite 프로젝트 폴더를 찾지 못했습니다.\n"
        "- /content/cstore_suite 또는 /content/drive/MyDrive/cstore_suite 위치를 확인하세요."
    )

os.chdir(proj)
print("📁 작업 폴더:", os.getcwd())
print("📂 포함 파일:", os.listdir())


✅ Python version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


OSError: [Errno 107] Transport endpoint is not connected

In [ ]:
# 2️⃣ requirements.txt 기반 의존성 설치
import os, subprocess, sys

req_path = os.path.join(os.getcwd(), "requirements.txt")
if not os.path.exists(req_path):
    print(f"⚠ requirements.txt 가 없어 기본 템플릿을 생성합니다: {req_path}")
    with open(req_path, "w", encoding="utf-8") as f:
        f.write("pandas==2.2.2\n")
        f.write("numpy==1.26.4\n")
        f.write("streamlit==1.39.0\n")
        f.write("altair>=5,<6\n")
        f.write("scikit-learn\n")
        f.write("pyngrok\n")
        f.write("cloudflared\n")
        f.write("xgboost\n")
        f.write("lightgbm\n")
        f.write("optuna\n")
else:
    print("📑 requirements.txt 위치:", req_path)

print("📦 의존성 설치 중... (다소 시간이 걸릴 수 있습니다)")
subprocess.run([sys.executable, "-m", "pip", "install", "-r", req_path], check=True)
print("✅ 의존성 설치 완료")


OSError: [Errno 107] Transport endpoint is not connected

In [ ]:
# 3️⃣ 모델 자동 학습 (models/best_model.pkl 이 없을 때만 수행)
import os, sys, glob, subprocess

proj = os.getcwd()
models_dir = os.path.join(proj, "models")
os.makedirs(models_dir, exist_ok=True)
best_model_path = os.path.join(models_dir, "best_model.pkl")

if os.path.exists(best_model_path):
    print("✅ 이미 학습된 모델이 있습니다:", best_model_path)
else:
    data_dir = os.path.join(proj, "data")
    if not os.path.isdir(data_dir):
        raise FileNotFoundError(f"data 폴더를 찾지 못했습니다: {data_dir}")

    candidates = [p for p in glob.glob(os.path.join(data_dir, "*.csv")) if os.path.isfile(p)]
    if not candidates:
        raise FileNotFoundError(f"data 폴더에서 CSV 파일을 찾지 못했습니다: {data_dir}")

    data_path = None
    # 우선순위: sample_sales.csv → seoul_gyeonggi_with_demand.csv → 그 외 첫 번째
    for name in ["sample_sales.csv", "seoul_gyeonggi_with_demand.csv"]:
        cand = os.path.join(data_dir, name)
        if os.path.exists(cand):
            data_path = cand
            break
    if data_path is None:
        data_path = candidates[0]

    print("📊 학습에 사용할 데이터:", data_path)

    cmd = [
        sys.executable,
        "quick_train_runner.py",
        "--data", data_path,
        "--project", proj,
        "--valid_ratio", "0.2",
        "--use_optuna",
        "--optuna_trials", "10",
    ]
    print("🚀 학습 명령:", " ".join(cmd))
    subprocess.run(cmd, check=True)
    if os.path.exists(best_model_path):
        print("✅ 학습 및 모델 저장 완료:", best_model_path)
    else:
        print("⚠ quick_train_runner.py 실행은 끝났지만 best_model.pkl 을 찾지 못했습니다.")


✅ 이미 학습된 모델이 있습니다: /content/drive/MyDrive/cstore_suite/models/best_model.pkl


In [ ]:
# 4️⃣ Streamlit 앱 + ngrok 터널 실행
#  - 셀 실행 후 출력되는 Public URL 로 접속하면 앱 화면이 보입니다.

import os, sys, subprocess, time

from pyngrok import ngrok, conf

# 기존 터널 정리
try:
    ngrok.kill()
except Exception:
    pass

port = 8501

# 토큰 읽기 (환경변수 우선, 없으면 입력 받기)
token = os.environ.get("NGROK_AUTHTOKEN", "").strip()
if not token:
    try:
        token = input("NGROK_AUTHTOKEN 을 입력하세요(없으면 엔터): ").strip()
    except EOFError:
        token = ""

if token:
    conf.get_default().auth_token = token
else:
    print("⚠ NGROK_AUTHTOKEN 이 비어 있습니다. 비인증 모드는 제한/에러가 날 수 있습니다.")

# ngrok 터널 오픈
print("🌐 ngrok 터널을 여는 중...")
tunnel = ngrok.connect(addr=f"http://localhost:{port}", proto="http")
public_url = tunnel.public_url
print("🌐 Public URL:", public_url)
print("➡ 위 URL 을 새 탭에서 열면 Streamlit 앱에 접속할 수 있습니다.")
print("(셀을 멈추면 터널도 종료됩니다.)")

# Streamlit 앱 실행 (로그는 아래에 출력)
cmd = [
    sys.executable,
    "-m", "streamlit",
    "run", "app_streamlit_pro.py",
    "--server.port", str(port),
    "--server.address", "0.0.0.0",
]
print("🚀 Streamlit 실행:", " ".join(cmd))
subprocess.run(cmd, check=True)


NGROK_AUTHTOKEN 을 입력하세요(없으면 엔터): 34Ug4uB0dodqJJVE41prai7dVdp_48R3SS6GGeSabkZFP89Xw
🌐 ngrok 터널을 여는 중...
🌐 Public URL: https://debra-didactic-preculturally.ngrok-free.dev
➡ 위 URL 을 새 탭에서 열면 Streamlit 앱에 접속할 수 있습니다.
(셀을 멈추면 터널도 종료됩니다.)
🚀 Streamlit 실행: /usr/bin/python3 -m streamlit run app_streamlit_pro.py --server.port 8501 --server.address 0.0.0.0
